In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import *
from resnet_zoo import *
from dataset import Dataset_per_file, Dataset_whole, Dataset_ori
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.autograd import Variable
import os
import numpy as np
from tqdm import tqdm
import argparse
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc
import sys
import os

from collections import OrderedDict


In [16]:
def test_epoch(PPG_model, test_loader, baseline=False):
    with torch.no_grad():
       

        PPG_preds = None
        PPG_pred_probs = None
        all_targets = None
        
        PPG_model.eval()
        tstart = datetime.now()

        for batch_idx, (PPG, target) in enumerate(test_loader):

            PPG = PPG.cuda().float()
            target = target.cuda().long()
            
            if baseline:
                _, PPG_out = model(PPG, PPG)
            else:
                _, _, _, _, _, PPG_out = model(PPG, PPG)
            
            PPG_predicted = PPG_out.argmax(1)
            PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

            if PPG_preds == None:
                PPG_pred_probs = PPG_predicted_prob
                PPG_preds = PPG_predicted
                all_targets = target
            else:
                PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                all_targets = torch.cat((all_targets, target))
        tend = datetime.now()

        precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
        pr_auc = auc(recall, precision)

        print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')


# SimSiam

In [17]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/resnet_50_epoch_40_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_1024_dim2_256_pred_True_splitheads/model_3.pt'

In [18]:
model = Res50SimSiam(1024, 256, predictor=True).cuda()

In [19]:
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for f in sorted(os.listdir(test_data_folder)):
    if '2400' in f and 'sully' not in f:
        
        x_path = test_data_folder + f
        y_path = test_data_folder + 'label_'+'_'.join(f[:-4].split('_')[1:-1]) + '.npy'

        print(f[:-4])
        print(x_path)
        print(y_path)
        test_dataset = Dataset_ori(x_path, y_path)
        testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

        test_epoch(model, testloader)

        print()

data_simband_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.7221
[TEST] 	PPG ROC AUC: 0.9051
[TEST] 	PPG PR  AUC: 0.8438

data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_badquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_badquality.npy
[TEST] 	PPG      F1: 0.6973
[TEST] 	PPG ROC AUC: 0.87
[TEST] 	PPG PR  AUC: 0.8228

data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_goodquality.npy
[TEST] 	PPG      F1: 0.9167
[TEST] 	PPG ROC AUC: 0.9977
[TEST] 	PPG PR  AUC: 0.991

data_staford_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_staford.npy
[TEST] 	PPG      F1: 0.5937
[TEST] 	PPG ROC AUC: 0.8545
[TEST] 	PPG PR  AUC: 0.6906

data_staford_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/